# Train

 - reference
  - https://www.kaggle.com/code/hannes82/pppm-deberta-v3-large-inference-cv-0-829
  
  - https://www.kaggle.com/code/leehann/inference-bert-for-uspatents
  
  - https://www.kaggle.com/code/diamondlee/u-s-pppm-simple-ensemble-baseline
  
  - https://www.kaggle.com/code/renokan/pppm-deberta-v3-large-additional-fold-s
  
  - https://www.kaggle.com/code/skloveyyp/2-deberta-1-roberta-analysis-and-using
  
  - https://www.kaggle.com/code/gaozhao/a-simple-ensemble-of-two
  
  - https://www.kaggle.com/code/luaubrey/uspppm-inference

## 1. Deberta-v3-large
- ### train.csv
 - Private Score : 0.8533 
 - Public Score : 0.8376
 
- ### train.csv / 5folds
 - Private Score : 0.8476 
 - Public Score : 0.8395

- ### train_remove0.csv
 - Private Score : 0.8387
 - Public Score : 0.8275

- ### tokenizer : bert-base-uncased
- ### tokenizer : anferico/bert-for-patents
- ### tokenizer : roberta-large

## 2. Roberta-base
- ### train.csv
 - Private Score : 0.7760
 - Public Score : 0.7584

## 3. BERT
- ### train.csv / tokenizer : bert-multilingual-cased
 - Private Score : 0.7130 
 - Public Score : 0.7076

## 4. Deberta v3 large + Roberta-large
- ### train.csv / 66 : 33
 - Private Score : 0.8545 
 - Public Score : 0.8390

- ### train_aug1.csv / 66 : 33
 - Private Score : 0.8548 
 - Public Score : 0.8384

- ### train_aug1.csv / 77 : 22
 - Private Score : 0.8549 
 - Public Score : 0.8377
 
- ### train_aug1.csv / 50 : 50
 - Private Score : 0.8521
 - Public Score : 0.8369
 
- ### train.csv / Deberta v3 large(5folds) + Roberta-large / 66 : 33
 - Private Score : 0.3692
 - Public Score : 0.3737

## 5. Deberta v3 large + Roberta-large + patent-BERT
- ### train.csv / 33 : 33 : 33
 - Private Score : 0.7732
 - Public Score : 0.7643

- ### train.csv / 60 : 20 : 20
 - Private Score : 0.8331
 - Public Score : 0.8205
 
- ### train.csv / 70 : 15 : 15
 - Private Score : 0.8432
 - Public Score : 0.8294

- ### train.csv / 60 : 10 : 30
 - Private Score : 0.7966
 - Public Score : 0.7852
 
- ### train.csv / 80 : 10 : 10
 - Private Score : 0.8488
 - Public Score : 0.8338

## 6. Deberta v3 large(baseline) + Deberta v3 large(BERT for usPatent) + Roberta-large
- ### train.csv / 33 : 33 : 33
 - Private Score : 0.8556
 - Public Score : 0.8431

## 7. Deberta v3 large(baseline) + Deberta v3 large(BERT for usPatent) 
- ### train.csv / 50 : 50
 - Private Score : 0.8549
 - Public Score : 0.8422

## 7. Deberta v3 large + Electra
- ### train.csv / Deberta v3 large(5folds) + Electra(2folds) / 60 : 40
 - Private Score : 0.8083
 - Public Score : 0.7884

- ### train_remove_token.csv / Deberta v3 large(5folds) + Electra(4folds) / 60 : 40
 - Private Score : 0.8122
 - Public Score : 0.7998

- ### train_remove_token.csv / Deberta v3 large(5folds) + Electra(2folds-10epoch) / 60 : 40
 - Private Score : 0.7952
 - Public Score : 0.7834

## 8. Deberta v3 large(baseline) + Deberta v3 large(BERT for usPatent) + BERT + Roberta-large
- ### train.csv / all models(5folds) / 50 : 30 : 10 : 10
 - Private Score : 0.8577
 - Public Score : 0.8458
 
- ### train.csv / all models(5folds) / 60 : 20 : 10 : 10
 - Private Score : 0.8571
 - Public Score : 0.8447
 
- ### train.csv / all models(5folds) / 60 : 30 : 0 : 10
 - Private Score : 0.8573
 - Public Score : 0.8449
 
- ### train.csv / all models(5folds) / 70 : 20 : 0 : 10
 - Private Score : 0.8563
 - Public Score : 0.8434
 
- ### train.csv / all models(5folds) / 60 : 30 : 5 : 5
 - Private Score : 0.8573
 - Public Score : 0.8448
 
- ### train.csv / all models(5folds) / 60 : 20 : 5 : 5
 - Private Score : 0.8568
 - Public Score : 0.8440

# Data Preprocessing

## 1. Remove data with score 0

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
file = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')

data=[]
for idx in range(len(file)):
    row=[]
    if file['score'][idx]==0.0:
        continue
    else:
        row.append(file['id'][idx])
        row.append(file['anchor'][idx])
        row.append(file['target'][idx])
        row.append(file['context'][idx])
        row.append(file['score'][idx])
        data.append(row)

In [ ]:
df = pd.DataFrame(data, columns=['id','anchor', 'target', 'context', 'score'])
df.to_csv("./train_remove0.csv", mode='w')

## 2. Generate score 1 data by creating the same data as 'anchor' and 'target'

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
file = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')

data=[]
anchor=''
same=0
for idx in range(len(file['anchor'])): 
    row=[]

    if anchor==file['anchor'][idx]:
        
        if anchor==file['target'][idx]:  
            same=1
            
    if anchor != file['anchor'][idx]:
        if same==0 and anchor!='':
            row.append(file['id'][idx])
            row.append(anchor)
            row.append(anchor)
            row.append(file['context'][idx])
            row.append(np.float64(1))
            data.append(row)
            row=[]      
        same=0
        anchor=file['anchor'][idx]
        
    row.append(file['id'][idx])
    row.append(file['anchor'][idx])
    row.append(file['target'][idx])
    row.append(file['context'][idx])
    row.append(file['score'][idx])
    data.append(row)

In [ ]:
df = pd.DataFrame(data, columns=['id','anchor', 'target', 'context', 'score'])
df.to_csv("./train_aug1.csv", mode='w')

## 3. Remove data with 7 or more tokens in 'target'

In [ ]:
import pandas as pd 

In [ ]:
file = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')

data=[]

for idx in range(len(file)):
    row=[]
    if len(file['target'][idx].split())<7:
        row.append(file['id'][idx])
        row.append(file['anchor'][idx])
        row.append(file['target'][idx])
        row.append(file['context'][idx])
        row.append(file['score'][idx])
        data.append(row)

In [ ]:
df = pd.DataFrame(data, columns=['id','anchor', 'target', 'context', 'score'])
df.to_csv("./train_remove_token.csv", mode='w')